In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip = True)

t_set = train_datagen.flow_from_directory("/kaggle/input/gtsrb-german-traffic-sign/Train",
                                          target_size = (64,64),
                                          batch_size=128,
                                          class_mode='categorical')

In [ ]:
# with tpu_strategy.scope():
cnn = Sequential()

#first Layer of cnn
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size = 2,strides =1))
cnn.add(Dropout(0.25))

#second Layer of CNN
cnn.add(Conv2D(filters = 128,kernel_size = 3 ,activation = 'relu'))
cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size = 2,strides =1))
cnn.add(Dropout(0.25))

#flatten
cnn.add(Flatten())

#full connection
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dropout(0.25))


#OUTPUT layer
cnn.add(Dense(units =43,activation = 'softmax'))

#Training the cnn
cnn.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
#Early Stopping
# Define early stopping
early_stopping = EarlyStopping(
    monitor='accuracy', # Which metric to monitor
    min_delta=0.01, # Minimum change to qualify as an improvement
    patience=2, # How many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
histroy = cnn.fit(x=t_set,epochs =3,batch_size=100,callbacks=[early_stopping])

In [ ]:
loss,accuracy = cnn.evaluate(t_set)
print(f"test accuracy:{accuracy*100:.2f}%")

In [ ]:
test_image = image.load_img('/kaggle/input/gtsrb-german-traffic-sign/Test/00060.png', target_size = (64, 64))
test_image

In [ ]:
class_label ={ 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }



test_image = image.load_img('/kaggle/input/gtsrb-german-traffic-sign/Test/00060.png', target_size = (64, 64))
print(test_image)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0)
aa =t_set.class_indices

print(aa)


prediction_index = np.argmax(result)
prediction = None

value=""
for key, value in aa.items():
    
    if value == prediction_index: 
        prediction = value
        keyy = int(key)
        print("Prediction:",class_label[keyy])
        break


In [ ]:

# class_indices = t_set.class_indices

test_folder_path = '/kaggle/input/gtsrb-german-traffic-sign/Test'

# Loop through each image in the folder
for img_name in os.listdir(test_folder_path):
    img_path = os.path.join(test_folder_path, img_name)
    
    # Check if the item is an image file
    if img_name.endswith(".png") or img_name.endswith(".jpg"):
        # Load and preprocess the image
        test_image = image.load_img(img_path, target_size=(64, 64))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)

        # Normalize and make prediction
        result = cnn.predict(test_image / 255.0)
        
        # Get the predicted class index
        prediction_index = np.argmax(result)

        # Map the prediction index to the corresponding class label
        for key, value in aa.items():
            if value == prediction_index:
                keyy = int(key)
                
                # Display the image
                img = image.load_img(img_path)
                plt.imshow(img)
                plt.axis('off')
                plt.title(f"Prediction: {class_label[keyy]}")
                plt.show()
                
                print(f"Prediction for {img_name}:\033[1m {class_label[keyy]}\033[0m")
                print("===========================================================")
                
                break

                